In [28]:
from qutip import *
from math import sqrt, cos, sin
import numpy as np
from gen_state import make_W_list
from rho_methods import compute_witnesses, get_rho
import matplotlib.pyplot as plt
from tqdm import tqdm

In [29]:
H = basis(2,0) # ket 1
V = basis(2,1) # ket 2
D = 1/sqrt(2) * (H + V)
A = 1/sqrt(2) * (H - V)
R = 1/sqrt(2)*(H + 1j*V)
L = 1/sqrt(2)*(H - 1j*V)


In [30]:
def pauli_state(W_mat):
    """ Checks which pauli matrix combos make up the witness, W_mat

        0 - IxI 4 - XxI 8 - YxI 12 - ZxI
        1 - IxX 5 - XxX 9 - YxX 13 - ZxX
        2 - IxY 6 - XxY 10 - YxY 14 - ZxY
        3 - IxZ 7 - XxZ 11 - YxZ 15 - ZxZ
    """

    II = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]).flatten()
    IX = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,0,1,0]]).flatten()
    IY = np.array([[0,-1j,0,0],[1j,0,0,0],[0,0,0,-1j],[0,0,1j,0]]).flatten()
    IZ = np.array([[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]]).flatten()

    XI = np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]]).flatten()
    XX = np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]]).flatten()
    XY = np.array([[0,0,0,-1j],[0,0,1j,0],[0,-1j,0,0],[1j,0,0,0]]).flatten()
    XZ = np.array([[0,0,1,0],[0,0,0,-1],[1,0,0,0],[0,-1,0,0]]).flatten()

    YI = np.array([[0,0,-1j,0],[0,0,0,-1j],[1j,0,0,0],[0,1j,0,0]]).flatten()
    YX = np.array([[0,0,0,-1j],[0,0,1j,0],[0,1j,0,0],[1j,0,0,0]]).flatten()
    YY = np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]]).flatten()
    YZ = np.array([[0,0,-1j,0],[0,0,0,1j],[1j,0,0,0],[0,-1j,0,0]]).flatten()

    ZI = np.array([[1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,-1]]).flatten()
    ZX = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,-1],[0,0,-1,0]]).flatten()
    ZY = np.array([[0,-1j,0,0],[1j,0,0,0],[0,0,0,1j],[0,0,-1j,0]]).flatten()
    ZZ = np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]]).flatten()
    
    W_flat = W_mat.flatten() # for some reason this doesn't work as desired ... 4x4 matrix
    
    pauli_mats =  np.array([II,IX,IY,IZ,XI,XX,XY,XZ,YI,YX,YY,YZ,ZI,ZX,ZY,ZZ])
    contains_pauli = [False]*16

    for i in range(len(pauli_mats)):
        prod = np.dot(W_flat,pauli_mats[i])
        if prod != 0:
            contains_pauli[i] = True
    return contains_pauli

In [31]:
def scan_wit(scan_params):
    group_count = 0
    groups = {}
    
    for params in scan_params:
        alpha, beta, theta, phi, chi, eta, lam, gamma = params
    
        # Calculate W_state most general
        W_state = tensor(np.cos(theta)*H + np.sin(theta)*np.exp(1j*phi)*V, np.cos(alpha)*H + np.sin(alpha)*np.exp(1j*beta)*V) + tensor(-np.exp(-1j*chi)*np.sin(eta)*H + np.cos(eta)*V, -np.exp(-1j*lam)*np.sin(gamma)*H + np.cos(gamma)*V)
        # print(W_state)
        # Calculate W
        W = partial_transpose(W_state*W_state.dag(), [0,1])
        # print("W:")
        # print(W)
        # Extract Pauli matrices
        mat = W.full()
        
        pauli_mats = pauli_state(mat)
    
        # Store Pauli matrices in groups
        groups[group_count] = pauli_mats
        group_count += 1
        # if group_count < 10:
        #     print("Pauli_mats")
        #     print(pauli_mats)
    return groups

In [32]:
num_angles = 4

# Create 1D arrays for each parameter
alpha_L = np.linspace(0, np.pi, num_angles)
beta_L = np.linspace(0, 2*np.pi, num_angles)
theta_L = np.linspace(0, np.pi, num_angles)
phi_L = np.linspace(0, 2*np.pi, num_angles)
chi_L = np.linspace(0, 2*np.pi, num_angles)
eta_L = np.linspace(0, np.pi, num_angles)
lambda_L = np.linspace(0, 2*np.pi, num_angles)
gamma_L = np.linspace(0, np.pi, num_angles)

# Create mesh grids for each parameter
mesh_grids = np.meshgrid(alpha_L, beta_L, theta_L, phi_L, chi_L, eta_L, lambda_L, gamma_L, indexing='ij')

# Stack the mesh grids along the last axis to get a 6x6x6x6 mesh grid
scan_list = np.stack(mesh_grids, axis=-1).reshape(-1, 8)


In [33]:
groups = scan_wit(scan_list)

In [34]:
# now we check which states have the least measurments 
# if the measurments are already covered by a W or Wp group alone, get rid of them. Otherwise,
# add to a new dictionary
sorted_groups = {}

for wit in range(len(scan_list)):
    wp_meas = [groups[wit][11], groups[wit][14], groups[wit][7],groups[wit][13],groups[wit][6],groups[wit][9]]
    # change this to match what's in notebook

    
    # here it can get tricky, because we filter based on W'
    # option 1: make it so we elim if it is doing two W' measures and something else
    # option 2: check if it is doing at least one and add all to the count anyways
    if (groups[wit][11] and groups[wit][14]) or (groups[wit][7] and groups[wit][13]) \
        or (groups[wit][6] and groups[wit][9]):
        num_measurements = sum([1 for meas in wp_meas if meas]) - 2
            
        sorted_groups[wit] = num_measurements

sorted_groups = {k: v for k, v in sorted(sorted_groups.items(), key=lambda item: item[1])}


In [26]:
# keep only the ones that require 1 or two extra measurement
min_sorted_groups = {k: v for k, v in sorted_groups.items() if (v <= 2 and v > 0)}
print(len(min_sorted_groups))

16624


In [35]:
# Initialize ortho_min_groups
ortho_min_groups = {wit: min_sorted_groups[wit] for wit in min_sorted_groups}

# Initialize W_arrays for all witness indices
W_arrays = {}

# Calculate and store W_arrays for each witness index
for wit in min_sorted_groups:
    alpha, beta, theta, phi, chi, eta, lam, gamma = scan_list[wit]
    # Calculate W_state most general
    W_state = tensor(np.cos(theta)*H + np.sin(theta)*np.exp(1j*phi)*V, np.cos(alpha)*H + np.sin(alpha)*np.exp(1j*beta)*V) + tensor(-np.exp(-1j*chi)*np.sin(eta)*H + np.cos(eta)*V, -np.exp(-1j*lam)*np.sin(gamma)*H + np.cos(gamma)*V)
    # Calculate W
    W = partial_transpose(W_state*W_state.dag(), [0,1])
    mat = W.full()
    W_arrays[wit] = np.array(mat).flatten()

# Iterate through witness indices
for wit in min_sorted_groups:
    W_array = W_arrays[wit]

    # Check orthogonality with other witness indices
    for curr_wit in min_sorted_groups:
        if wit == curr_wit:
            continue
        
        curr_W_array = W_arrays[curr_wit]
        
        # Check orthogonality using dot product
        if np.dot(curr_W_array, W_array) == 0:
            del ortho_min_groups[wit]
            break

In [36]:
print(len(ortho_min_groups))

838


In [138]:
# divide ortho_min_groups into subsets
num_sets = 10
set_size = len(ortho_min_groups)/ num_sets

ortho_min_groups_1 = {}
ortho_min_groups_2 = {}
ortho_min_groups_3 = {}
ortho_min_groups_4 = {}
ortho_min_groups_5 = {}
ortho_min_groups_6 = {}
ortho_min_groups_7 = {}
ortho_min_groups_8 = {}
ortho_min_groups_9 = {}
ortho_min_groups_10 = {}

for index, (key, value) in enumerate(ortho_min_groups.items()):
    if index < set_size:
        ortho_min_groups_1[key] = value
    elif index > set_size and index <= 2*set_size:
        ortho_min_groups_2[key] = value
    elif index > 2*set_size and index <= 3*set_size:
        ortho_min_groups_3[key] = value
    elif index > 3*set_size and index <= 4*set_size:
        ortho_min_groups_4[key] = value
    elif index > 4*set_size and index <= 5*set_size:
        ortho_min_groups_5[key] = value
    elif index > 5*set_size and index <= 6*set_size:
        ortho_min_groups_6[key] = value
    elif index > 6*set_size and index <= 7*set_size:
        ortho_min_groups_7[key] = value
    elif index > 7*set_size and index <= 8*set_size:
        ortho_min_groups_8[key] = value
    elif index > 8*set_size and index <= 9*set_size:
        ortho_min_groups_9[key] = value
    elif index > 9*set_size and index <= 10*set_size:
        ortho_min_groups_10[key] = value
        

In [139]:
full_num_witnessed = {}

In [177]:
# Initialize num_witnessed dictionary
num_witnessed_10 = {k: 0 for k, v in ortho_min_groups_10.items()}

# Define the number of intervals
num_intervals = 5

# Generate mesh grids for each parameter
alpha_L, beta_L, theta_L, phi_L, chi_L = np.meshgrid(np.linspace(0, np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      np.linspace(0, np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      indexing='ij')

# Flatten mesh grids
alpha_vals = alpha_L.flatten()
beta_vals = beta_L.flatten()
theta_vals = theta_L.flatten()
phi_vals = phi_L.flatten()
chi_vals = chi_L.flatten()



# Iterate over all combinations of parameters
for alpha, beta, theta, phi, chi in tqdm(zip(alpha_vals, beta_vals, theta_vals, phi_vals, chi_vals)):
    # Calculate entangled state
    ent_state = tensor(np.cos(theta)*H + np.sin(theta)*np.exp(1j*phi)*V,
                       np.cos(alpha)*H + np.sin(alpha)*np.exp(1j*beta)*V) + \
                np.exp(1j*chi) * tensor(-np.exp(-1j*phi)*np.sin(theta)*H + np.cos(theta)*V,
                                         -np.exp(-1j*beta)*np.sin(alpha)*H + np.cos(alpha)*V)
    rho = ket2dm(ent_state)
    # print(num_witnessed)
    # Iterate over witness indices
    for wit in ortho_min_groups_10:
        params = scan_list[wit]

        # Calculate W_state
        W_state = tensor(np.cos(params[2])*H + np.sin(params[2])*np.exp(1j*params[3])*V,
                         np.cos(params[0])*H + np.sin(params[0])*np.exp(1j*params[1])*V) + \
                  np.exp(1j*params[4]) * tensor(-np.exp(-1j*params[3])*np.sin(params[2])*H + np.cos(params[2])*V,
                                                 -np.exp(-1j*params[1])*np.sin(params[0])*H + np.cos(params[0])*V)
        
        # Calculate W
        W = partial_transpose(W_state*W_state.dag(), [0, 1])

        # Calculate expectation value
        exp_val = (W*rho).tr()
        # print(exp_val, wit)
        if np.real(exp_val) < 0:
            num_witnessed_10[wit] += 1
                    
                    
                
                
                

3125it [05:51,  8.89it/s]


In [178]:
print(num_witnessed_10)

{59437: 466, 59601: 466, 59602: 466, 59613: 466, 59614: 466, 59617: 466, 59629: 466, 60177: 466, 60178: 466, 60189: 466, 60190: 466, 60193: 466, 60205: 466, 60369: 466, 60370: 466, 60381: 466, 60382: 466, 60385: 466, 60397: 466, 62481: 466, 62493: 466, 62673: 466, 62685: 466, 63249: 466, 63261: 466, 63441: 466, 63453: 466, 63505: 466, 63506: 466, 63517: 466, 63518: 466, 63521: 466, 63533: 466, 63697: 466, 63698: 466, 63709: 466, 63710: 466, 63713: 466, 63725: 466, 64273: 466, 64274: 466, 64285: 466, 64286: 466, 64289: 466, 64301: 466, 64465: 466, 64466: 466, 64477: 466, 64478: 466, 64481: 466, 64493: 466, 1562: 588, 1754: 588, 5658: 588, 5850: 588, 9754: 588, 9946: 588, 13850: 588, 14042: 588, 18953: 510, 19017: 687, 19081: 551, 19145: 510, 24737: 551, 24749: 551, 24993: 551, 25005: 551, 25249: 551, 25261: 551, 25505: 551, 25517: 551, 31241: 510, 31305: 687, 31369: 551, 31433: 510, 42128: 551, 42132: 551, 42136: 551, 42140: 551, 42896: 551, 42900: 551, 42904: 551, 42908: 551}


In [179]:
for k,v in num_witnessed_10.items():
    if k not in full_num_witnessed:
        full_num_witnessed[k] = v



In [180]:
print(len(full_num_witnessed))

838


In [188]:
print(full_num_witnessed)

# {1057: 466, 1069: 466, 1249: 466, 1261: 466, 1825: 466, 1837: 466, 2017: 466, 2029: 466, 2081: 466, 2093: 466, 2273: 466, 2285: 466, 2849: 466, 2861: 466, 3041: 466, 3053: 466, 5153: 466, 5165: 466, 5345: 466, 5357: 466, 5921: 466, 5933: 466, 6113: 466, 6125: 466, 6177: 466, 6189: 466, 6369: 466, 6381: 466, 6945: 466, 6957: 466, 7137: 466, 7149: 466, 9249: 466, 9261: 466, 9441: 466, 9453: 466, 10017: 466, 10029: 466, 10209: 466, 10221: 466, 10273: 466, 10285: 466, 10465: 466, 10477: 466, 11041: 466, 11053: 466, 11233: 466, 11245: 466, 13345: 466, 13357: 466, 13537: 466, 13549: 466, 14113: 466, 14125: 466, 14305: 466, 14317: 466, 14369: 466, 14381: 466, 14561: 466, 14573: 466, 15137: 466, 15149: 466, 15329: 466, 15341: 466, 16386: 466, 16398: 466, 16401: 466, 16402: 466, 16413: 466, 16414: 466, 16450: 569, 16462: 569, 16514: 551, 16526: 551, 16578: 466, 16590: 466, 16593: 466, 16594: 466, 16605: 466, 16606: 466, 16642: 466, 16654: 466, 16657: 466, 16658: 466, 16669: 466, 16670: 466, 16706: 569, 16718: 569, 16770: 551, 16782: 551, 16834: 466, 16846: 466, 16849: 466, 16850: 466, 16861: 466, 16862: 466, 16898: 466, 16910: 466, 16913: 466, 16914: 466, 16925: 466, 16926: 466, 16962: 569, 16974: 569, 17026: 551, 17038: 551, 17090: 466, 17102: 466, 17105: 466, 17106: 466, 17117: 466, 17118: 466, 17154: 466, 17166: 466, 17169: 466, 17170: 466, 17181: 466, 17182: 466, 17218: 569, 17230: 569, 17282: 551, 17294: 551, 17346: 466, 17358: 466, 17361: 466, 17362: 466, 17373: 466, 17374: 466, 17427: 444, 17431: 444, 17435: 444, 17439: 444, 17441: 444, 17442: 444, 17443: 444, 17447: 444, 17451: 444, 17453: 444, 17454: 444, 17455: 444, 17619: 444, 17623: 444, 17627: 444, 17631: 444, 17633: 444, 17634: 444, 17635: 444, 17639: 444, 17643: 444, 17645: 444, 17646: 444, 17647: 444, 18195: 444, 18199: 444, 18203: 444, 18207: 444, 18209: 444, 18210: 444, 18211: 444, 18215: 444, 18219: 444, 18221: 444, 18222: 444, 18223: 444, 18387: 444, 18391: 444, 18395: 444, 18399: 444, 18401: 444, 18402: 444, 18403: 444, 18407: 444, 18411: 444, 18413: 444, 18414: 444, 18415: 444, 18449: 466, 18461: 466, 18641: 466, 18653: 466, 19217: 466, 19229: 466, 19409: 466, 19421: 466, 19473: 466, 19485: 466, 19489: 466, 19501: 466, 19506: 466, 19518: 466, 19570: 569, 19582: 569, 19634: 551, 19646: 551, 19665: 466, 19677: 466, 19681: 466, 19693: 466, 19698: 466, 19710: 466, 19729: 466, 19741: 466, 19745: 466, 19757: 466, 19762: 466, 19774: 466, 19826: 569, 19838: 569, 19890: 551, 19902: 551, 19921: 466, 19933: 466, 19937: 466, 19949: 466, 19954: 466, 19966: 466, 19985: 466, 19997: 466, 20001: 466, 20013: 466, 20018: 466, 20030: 466, 20082: 569, 20094: 569, 20146: 551, 20158: 551, 20177: 466, 20189: 466, 20193: 466, 20205: 466, 20210: 466, 20222: 466, 20241: 466, 20253: 466, 20257: 466, 20269: 466, 20274: 466, 20286: 466, 20338: 569, 20350: 569, 20402: 551, 20414: 551, 20433: 466, 20445: 466, 20449: 466, 20461: 466, 20466: 466, 20478: 466, 22122: 544, 23130: 627, 27030: 627, 28674: 466, 28686: 466, 28689: 466, 28690: 466, 28701: 466, 28702: 466, 28738: 569, 28750: 569, 28802: 551, 28814: 551, 28866: 466, 28878: 466, 28881: 466, 28882: 466, 28893: 466, 28894: 466, 28930: 466, 28942: 466, 28945: 466, 28946: 466, 28957: 466, 28958: 466, 28994: 569, 29006: 569, 29058: 551, 29070: 551, 29122: 466, 29134: 466, 29137: 466, 29138: 466, 29149: 466, 29150: 466, 29186: 466, 29198: 466, 29201: 466, 29202: 466, 29213: 466, 29214: 466, 29250: 569, 29262: 569, 29314: 551, 29326: 551, 29378: 466, 29390: 466, 29393: 466, 29394: 466, 29405: 466, 29406: 466, 29442: 466, 29454: 466, 29457: 466, 29458: 466, 29469: 466, 29470: 466, 29506: 569, 29518: 569, 29570: 551, 29582: 551, 29634: 466, 29646: 466, 29649: 466, 29650: 466, 29661: 466, 29662: 466, 29715: 444, 29719: 444, 29723: 444, 29727: 444, 29729: 444, 29730: 444, 29731: 444, 29735: 444, 29739: 444, 29741: 444, 29742: 444, 29743: 444, 29907: 444, 29911: 444, 29915: 444, 29919: 444, 29921: 444, 29922: 444, 29923: 444, 29927: 444, 29931: 444, 29933: 444, 29934: 444, 29935: 444, 30483: 444, 30487: 444, 30491: 444, 30495: 444, 30497: 444, 30498: 444, 30499: 444, 30503: 444, 30507: 444, 30509: 444, 30510: 444, 30511: 444, 30675: 444, 30679: 444, 30683: 444, 30687: 444, 30689: 444, 30690: 444, 30691: 444, 30695: 444, 30699: 444, 30701: 444, 30702: 444, 30703: 444, 30737: 466, 30749: 466, 30929: 466, 30941: 466, 31505: 466, 31517: 466, 31697: 466, 31709: 466, 31761: 466, 31773: 466, 31777: 466, 31789: 466, 31794: 466, 31806: 466, 31858: 569, 31870: 569, 31922: 551, 31934: 551, 31953: 466, 31965: 466, 31969: 466, 31981: 466, 31986: 466, 31998: 466, 32017: 466, 32029: 466, 32033: 466, 32045: 466, 32050: 466, 32062: 466, 32114: 569, 32126: 569, 32178: 551, 32190: 551, 32209: 466, 32221: 466, 32225: 466, 32237: 466, 32242: 466, 32254: 466, 32273: 466, 32285: 466, 32289: 466, 32301: 466, 32306: 466, 32318: 466, 32370: 569, 32382: 569, 32434: 551, 32446: 551, 32465: 466, 32477: 466, 32481: 466, 32493: 466, 32498: 466, 32510: 466, 32529: 466, 32541: 466, 32545: 466, 32557: 466, 32562: 466, 32574: 466, 32626: 569, 32638: 569, 32690: 551, 32702: 551, 32721: 466, 32733: 466, 32737: 466, 32749: 466, 32754: 466, 32766: 466, 32769: 466, 32781: 466, 32786: 466, 32798: 466, 32833: 551, 32845: 551, 32897: 551, 32909: 551, 32961: 466, 32973: 466, 32978: 466, 32990: 466, 33025: 466, 33037: 466, 33042: 466, 33054: 466, 33089: 551, 33101: 551, 33153: 551, 33165: 551, 33217: 466, 33229: 466, 33234: 466, 33246: 466, 33281: 466, 33293: 466, 33298: 466, 33310: 466, 33345: 551, 33357: 551, 33409: 551, 33421: 551, 33473: 466, 33485: 466, 33490: 466, 33502: 466, 33537: 466, 33549: 466, 33554: 466, 33566: 466, 33601: 551, 33613: 551, 33665: 551, 33677: 551, 33729: 466, 33741: 466, 33746: 466, 33758: 466, 33811: 466, 33815: 466, 33819: 466, 33823: 466, 34003: 466, 34007: 466, 34011: 466, 34015: 466, 34579: 466, 34583: 466, 34587: 466, 34591: 466, 34771: 466, 34775: 466, 34779: 466, 34783: 466, 34833: 444, 34845: 444, 35025: 444, 35037: 444, 35601: 444, 35613: 444, 35793: 444, 35805: 444, 35858: 466, 35870: 466, 35889: 466, 35901: 466, 35953: 551, 35965: 551, 36017: 551, 36029: 551, 36050: 466, 36062: 466, 36081: 466, 36093: 466, 36114: 466, 36126: 466, 36145: 466, 36157: 466, 36209: 551, 36221: 551, 36273: 551, 36285: 551, 36306: 466, 36318: 466, 36337: 466, 36349: 466, 36370: 466, 36382: 466, 36401: 466, 36413: 466, 36465: 551, 36477: 551, 36529: 551, 36541: 551, 36562: 466, 36574: 466, 36593: 466, 36605: 466, 36626: 466, 36638: 466, 36657: 466, 36669: 466, 36721: 551, 36733: 551, 36785: 551, 36797: 551, 36818: 466, 36830: 466, 36849: 466, 36861: 466, 38505: 627, 42405: 627, 43413: 544, 45057: 466, 45069: 466, 45074: 466, 45086: 466, 45121: 551, 45133: 551, 45185: 551, 45197: 551, 45249: 466, 45261: 466, 45266: 466, 45278: 466, 45313: 466, 45325: 466, 45330: 466, 45342: 466, 45377: 551, 45389: 551, 45441: 551, 45453: 551, 45505: 466, 45517: 466, 45522: 466, 45534: 466, 45569: 466, 45581: 466, 45586: 466, 45598: 466, 45633: 551, 45645: 551, 45697: 551, 45709: 551, 45761: 466, 45773: 466, 45778: 466, 45790: 466, 45825: 466, 45837: 466, 45842: 466, 45854: 466, 45889: 551, 45901: 551, 45953: 551, 45965: 551, 46017: 466, 46029: 466, 46034: 466, 46046: 466, 46099: 466, 46103: 466, 46107: 466, 46111: 466, 46291: 466, 46295: 466, 46299: 466, 46303: 466, 46867: 466, 46871: 466, 46875: 466, 46879: 466, 47059: 466, 47063: 466, 47067: 466, 47071: 466, 47121: 444, 47133: 444, 47313: 444, 47325: 444, 47889: 444, 47901: 444, 48081: 444, 48093: 444, 48146: 466, 48158: 466, 48177: 466, 48189: 466, 48241: 551, 48253: 551, 48305: 551, 48317: 551, 48338: 466, 48350: 466, 48369: 466, 48381: 466, 48402: 466, 48414: 466, 48433: 466, 48445: 466, 48497: 551, 48509: 551, 48561: 551, 48573: 551, 48594: 466, 48606: 466, 48625: 466, 48637: 466, 48658: 466, 48670: 466, 48689: 466, 48701: 466, 48753: 551, 48765: 551, 48817: 551, 48829: 551, 48850: 466, 48862: 466, 48881: 466, 48893: 466, 48914: 466, 48926: 466, 48945: 466, 48957: 466, 49009: 551, 49021: 551, 49073: 551, 49085: 551, 49106: 466, 49118: 466, 49137: 466, 49149: 466, 50193: 466, 50205: 466, 50385: 466, 50397: 466, 50961: 466, 50973: 466, 51153: 466, 51165: 466, 51217: 466, 51218: 466, 51229: 466, 51230: 466, 51233: 466, 51245: 466, 51409: 466, 51410: 466, 51421: 466, 51422: 466, 51425: 466, 51437: 466, 51985: 466, 51986: 466, 51997: 466, 51998: 466, 52001: 466, 52013: 466, 52177: 466, 52178: 466, 52189: 466, 52190: 466, 52193: 466, 52205: 466, 54289: 466, 54301: 466, 54481: 466, 54493: 466, 55057: 466, 55069: 466, 55249: 466, 55261: 466, 55313: 466, 55314: 466, 55325: 466, 55326: 466, 55329: 466, 55341: 466, 55505: 466, 55506: 466, 55517: 466, 55518: 466, 55521: 466, 55533: 466, 56081: 466, 56082: 466, 56093: 466, 56094: 466, 56097: 466, 56109: 466, 56273: 466, 56274: 466, 56285: 466, 56286: 466, 56289: 466, 56301: 466, 58385: 466, 58397: 466, 58577: 466, 58589: 466, 59153: 466, 59165: 466, 59345: 466, 59357: 466, 59409: 466, 59410: 466, 59421: 466, 59422: 466, 59425: 466, 59437: 466, 59601: 466, 59602: 466, 59613: 466, 59614: 466, 59617: 466, 59629: 466, 60177: 466, 60178: 466, 60189: 466, 60190: 466, 60193: 466, 60205: 466, 60369: 466, 60370: 466, 60381: 466, 60382: 466, 60385: 466, 60397: 466, 62481: 466, 62493: 466, 62673: 466, 62685: 466, 63249: 466, 63261: 466, 63441: 466, 63453: 466, 63505: 466, 63506: 466, 63517: 466, 63518: 466, 63521: 466, 63533: 466, 63697: 466, 63698: 466, 63709: 466, 63710: 466, 63713: 466, 63725: 466, 64273: 466, 64274: 466, 64285: 466, 64286: 466, 64289: 466, 64301: 466, 64465: 466, 64466: 466, 64477: 466, 64478: 466, 64481: 466, 64493: 466, 1562: 588, 1754: 588, 5658: 588, 5850: 588, 9754: 588, 9946: 588, 13850: 588, 14042: 588, 18953: 510, 19017: 687, 19081: 551, 19145: 510, 24737: 551, 24749: 551, 24993: 551, 25005: 551, 25249: 551, 25261: 551, 25505: 551, 25517: 551, 31241: 510, 31305: 687, 31369: 551, 31433: 510, 42128: 551, 42132: 551, 42136: 551, 42140: 551, 42896: 551, 42900: 551, 42904: 551, 42908: 551}

{1057: 466, 1069: 466, 1249: 466, 1261: 466, 1825: 466, 1837: 466, 2017: 466, 2029: 466, 2081: 466, 2093: 466, 2273: 466, 2285: 466, 2849: 466, 2861: 466, 3041: 466, 3053: 466, 5153: 466, 5165: 466, 5345: 466, 5357: 466, 5921: 466, 5933: 466, 6113: 466, 6125: 466, 6177: 466, 6189: 466, 6369: 466, 6381: 466, 6945: 466, 6957: 466, 7137: 466, 7149: 466, 9249: 466, 9261: 466, 9441: 466, 9453: 466, 10017: 466, 10029: 466, 10209: 466, 10221: 466, 10273: 466, 10285: 466, 10465: 466, 10477: 466, 11041: 466, 11053: 466, 11233: 466, 11245: 466, 13345: 466, 13357: 466, 13537: 466, 13549: 466, 14113: 466, 14125: 466, 14305: 466, 14317: 466, 14369: 466, 14381: 466, 14561: 466, 14573: 466, 15137: 466, 15149: 466, 15329: 466, 15341: 466, 16386: 466, 16398: 466, 16401: 466, 16402: 466, 16413: 466, 16414: 466, 16450: 569, 16462: 569, 16514: 551, 16526: 551, 16578: 466, 16590: 466, 16593: 466, 16594: 466, 16605: 466, 16606: 466, 16642: 466, 16654: 466, 16657: 466, 16658: 466, 16669: 466, 16670: 466, 167

In [186]:
# exclude all the witnesses that are witnessing 0 things
# filtered_num_witnessed = {k: v for k, v in num_witnessed.items() if v > 0}

# print(filtered_num_witnessed)

# get the parameters / pauli mats for each mat that witnessed things
witness_params = {k:scan_list[k] for k,v in full_num_witnessed.items()}
witness_pauli_mats = {k:groups[k] for k,v in full_num_witnessed.items()}

# for wit in witness_pauli_mats:
#     if witness_pauli_mats[wit] != [True, True, False, True, True, True, False, True, False, True, True, False, True, True, False, True]\
#         and witness_pauli_mats[wit] != [True, True, True, True, True, True, True, True, False, True, True, False, True, True, False, True]\
#         and witness_pauli_mats[wit] != [True, True, False, True, True, True, True, True, True, True, True, False, True, True, False, True]:
#         print(wit)
#         print(witness_pauli_mats[wit])
# print(witness_pauli_mats)

# II,IX,IY,IZ,XI,XX,XY,XZ,YI,YX,YY,YZ,ZI,ZX,ZY,ZZ
# W'1-3 - II, ZZ, XX, YY, ZI, IZ, IX, XI, IY, YI, XY, YX
# W'4-6 - II, ZZ, XX, YY, ZI, IZ, IX, XI, IY, YI, YZ, ZY
# W'7-9 - II, ZZ, XX, YY, ZI, IZ, IX, XI, IY, YI, XZ, ZX

# [True, True, False, True, True, True, False, True, False, True, True, False, True, True, False, True] (all the wits in first set)
# 1 - W, XZ, ZX, YX 
# 23130,27030,38505,42405 [True, False, True, True, False, True, False, False, True, True, True, True, True, False, True, True]
# 2 - W, XY, YX, YZ, ZY
# 1562 [True, True, True, True, True, True, True, True, False, True, True, False, True, True, False, True]
# 3 - W, ZX, YX, XZ, XY
# 18953 [True, True, False, True, True, True, True, True, True, True, True, False, True, True, False, True]
# 4 - W, ZX, YX, XZ, XY


In [187]:
# step 1 - figure out which witnesses witness the most states in these 5 groups / if there is a correlation based on the groupings

num_witnessed_by_group = {1:[],2:[],3:[],4:[]}
new_wit_groups = {1:[],2:[],3:[],4:[]}

for wit in full_num_witnessed:
    if witness_pauli_mats[wit] == [True, True, False, True, True, True, False, True, False, True, True, False, True, True, False, True]:
        num_witnessed_by_group[1] += [full_num_witnessed[wit]]
        new_wit_groups[1] += [wit]
    elif witness_pauli_mats[wit] == [True, False, True, True, False, True, False, False, True, True, True, True, True, False, True, True]:
        num_witnessed_by_group[2] += [full_num_witnessed[wit]]
        new_wit_groups[2] += [wit]
    elif witness_pauli_mats[wit] == [True, True, True, True, True, True, True, True, False, True, True, False, True, True, False, True]:
        num_witnessed_by_group[3] += [full_num_witnessed[wit]]
        new_wit_groups[3] += [wit]
    elif witness_pauli_mats[wit] == [True, True, False, True, True, True, True, True, True, True, True, False, True, True, False, True]:
        num_witnessed_by_group[4] += [full_num_witnessed[wit]]
        new_wit_groups[4] += [wit]
    else:
        print("no group found")
print(num_witnessed_by_group)

# {1: [466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 544, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 444, 444, 444, 444, 444, 444, 444, 444, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 544, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 444, 444, 444, 444, 444, 444, 444, 444, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 551, 551, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466], 2: [627, 627, 627, 627], 3: [588, 588, 588, 588, 588, 588, 588, 588, 551, 551, 551, 551, 551, 551, 551, 551], 4: [510, 687, 551, 510, 551, 551, 551, 551, 551, 551, 551, 551, 510, 687, 551, 510]}

{1: [466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 466, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 444, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 466, 569, 569, 551, 551, 466, 466, 466, 466, 466, 

In [ ]:
# step 2 - group the witnesses based on W' measurement then additional measurement

In [ ]:
# step 3 - do a more comprehensive scan if time